In [1]:
import cv2
import numpy as np
import cv2.aruco as aruco
from operator import itemgetter, attrgetter

In [2]:
def getFrame(dicIds):
    """
    The function take the detected marckers in a dictionary format
    and returns the frame of the video
    """
    
    # get the upper boundary of the frame
    # sort the markers according to the Y-cor and take the last two markers
    top_Markers_pts = sorted(sorted({v[0] for v in markers.values()} , key=itemgetter(1))[-2:], reverse=True)
    # get the ids of these markers
    top_Markers_ids = [k for pt in top_Markers_pts for k, v in markers.items() if pt == v[0]]

    # sort the markers according to the Y-cor and take the first two markers
    bottom_Markers_pts = sorted(sorted({v[0] for v in markers.values()} , key=itemgetter(1))[0:2])
    # get the ids of these markers
    bottom_Markers_ids = [k for pt in bottom_Markers_pts for k, v in markers.items() if pt == v[0]]
    
    # combine sorted ids together
    sortedIds = bottom_Markers_ids + top_Markers_ids

    # return the border of the frame
    showingBorder = [markers[sortedIds[0]][0], markers[sortedIds[1]][1], 
                         markers[sortedIds[2]][2], markers[sortedIds[3]][3]]
    return showingBorder

In [3]:
# read video to show
myVid = cv2.VideoCapture('video.mp4')

# activate mobile camera
address = "http://192.168.0.9:8080/video"
cam = cv2.VideoCapture(0)
cam.open(address)

# create empty dictionary to save markers in 
markers = {}

# call ArUco marker dictionary
dictionary = aruco.Dictionary_get(aruco.DICT_6X6_250)
parameters = aruco.DetectorParameters_create()

while True:
    ret, frame = cam.read()
    
    # Detect the markers in the image
    corners, ids, rejected = cv2.aruco.detectMarkers(frame, dictionary, parameters=parameters)
    
    # check if the 4 corners of the marker is dected
    if len(corners) > 0:
        # flatten the ArUco IDs list
        ids = ids.flatten()

        # loop over the detected ArUCo corners
        for (markerCorners, markerID) in zip(corners, ids):
            # extract the marker corners (which are always returned in
            # top-right, top-left, bottom-left, and bottom-right order)
            corners = markerCorners.reshape((4, 2))
            
            # add detected pts to marker dictionary
            markers[markerID] = list(map(tuple, corners))
            
        # if the four markers are detcted call getFrame function to get the boundary of the showing area
        if len(markers) == 4:
            showingBorder = np.float32(getFrame(markers))    

            # mask the refrenced object
            markerArea = cv2.fillPoly(frame, np.int32([showingBorder]), (0, 0, 0), lineType=cv2.LINE_AA) 

            # read the video
            success, inputVid = myVid.read()            
            (hVideo, wVideo) = inputVid.shape[:2]

            # get frame border of the video to transform it to the marker area
            InputVidBorder = np.float32([[0, 0], [wVideo, 0], [wVideo, hVideo], [0, hVideo]])

            # Calculate transformation matrix
            matrix = cv2.getPerspectiveTransform(InputVidBorder, showingBorder)

            # warp video to the referenced marker
            warpVid = cv2.warpPerspective(inputVid, matrix, (frame.shape[1], frame.shape[0])) 
            output = cv2.add(markerArea, warpVid)

            # Display the resulting frame
            cv2.imshow('frame', output)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
    else:
        # Display the resulting frame
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
# When everything done, release the capture
cam.release()
cv2.destroyAllWindows()